In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input

input_file = '../assets/data/recommendation_dataset.csv'

# Charger les données
data = pd.read_csv(input_file, sep=';')

# Remplacer 99 par NaN
data.replace(99, np.nan, inplace=True)

# Séparer les features et les targets
X = data.iloc[:, 1:].values  # Les valeurs des restaurants
y = data.iloc[:, 1:].values  # Les mêmes valeurs, utilisé pour prédire

# Remplacer les NaN par la moyenne des colonnes
col_mean = np.nanmean(X, axis=0)
inds = np.where(np.isnan(X))  # Indices des NaN
X[inds] = np.take(col_mean, inds[1])  # Remplacement

print("Vérification des NaN dans les données d'entrée :")
print(np.isnan(X).sum())  # Compter les NaN dans X

print("Statistiques des données d'entrée :")
print(pd.DataFrame(X).describe())

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Construction du modèle
model = keras.Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Remplacez l'input_shape par Input
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(X_train.shape[1], activation='linear'))  # Sortie de la même dimension que l'entrée

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Prédire les valeurs manquantes
predictions = model.predict(X_test)

# Inverser la normalisation pour obtenir les valeurs d'origine
predictions_inverse = scaler.inverse_transform(predictions)

# Remplacer les NaN dans le dataset original avec les prédictions
for i in range(predictions_inverse.shape[0]):
    for j in range(predictions_inverse.shape[1]):
        if np.isnan(X_test[i][j]):
            X_test[i][j] = predictions_inverse[i][j]
            
# Créer un DataFrame pour les prédictions
predictions_df = pd.DataFrame(predictions_inverse, columns=data.columns[1:])  # Utiliser les mêmes noms de colonnes

# Enregistrer le DataFrame dans un fichier CSV
predictions_df.to_csv('predictions.csv', sep=';', index=False)

print("Les prédictions ont été enregistrées dans 'predictions.csv'.")



Vérification des NaN dans les données d'entrée :
0
Statistiques des données d'entrée :
                 0             1            2             3             4   \
count  73421.000000  73421.000000  73421.00000  73421.000000  73421.000000   
mean       0.903260      0.165138      0.19747     -1.415242      0.235663   
std        3.444031      3.880206      3.59860      3.384588      5.277781   
min       -9.950000     -9.950000     -9.95000     -9.950000     -9.950000   
25%        0.903260      0.165138      0.19747     -1.415242     -3.980000   
50%        0.903260      0.165138      0.19747     -1.415242      0.830000   
75%        0.903260      0.290000      0.19747     -1.415242      4.370000   
max        9.610000      9.370000      9.61000      9.370000      9.510000   

                 5             6             7             8             9   \
count  73421.000000  73421.000000  73421.000000  73421.000000  73421.000000   
mean       1.328924     -0.637033     -0.997142     

KeyboardInterrupt: 